In [ ]:
# Basic environment modules
import numpy as np
import pandas as pd
import psycopg2 as pg # PostgreSQL module
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style
from matplotlib import font_manager
import matplotlib.font_manager as fm

# Data analysis modules
from CoolProp.CoolProp import PropsSI, Props
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from scipy.integrate import ode
from scipy.interpolate import griddata
from sqlalchemy import create_engine

# User-defined libraries
from Model import *
from PhysicalProperty import *
from StructuredQuery import *
from Numeric import *
from Authentication import *

# Set the display format to show 6 decimal places
pd.options.display.float_format = '{:.6f}'.format

print("Import modules successfully.")

# load class instances
sql = StructuredQuery()
pro = PhysicalProperty()
mod = Model()
oau = Authentication()

In [ ]:
# Connect to DB Server and get data
# Load json file (Security)
json_parse = oau.get_apikey(json_filename='key.json')
json_res = pd.json_normalize(json_parse['sql'])

# Set query from DB
sql = "SELECT * FROM CHF.raw_database"
#sql ="SELECT * FROM CHF.raw_database WHERE refri IN ('R12', 'R113', 'R114', 'R21') AND idx != '31'"
#sql = "SELECT * FROM CHF.raw_database WHERE refri IN ('H2O', 'D2O') AND idx != '31'"
#sql = "SELECT * FROM CHF.raw_database WHERE idx = '12' AND tno = 'T6'"

# Connect DB server
#(conn, db_engine) = sql.connect(json_res["host"][1], json_res["dbname"][1], json_res["user"][1], json_res["port"][1], json_res["password"][1], json_res["service"][1]) #postgreSQL
connect = pys.connect(host=json_res["host"][1], user = json_res["user"][1], password = json_res["password"][1], cursorclass=pys.cursors.DictCursor)
cur = connect.cursor()
cur.execute(sql)


result = cur.fetchall()
raw_tb = pd.DataFrame(result)

del result, sql

# Set query from DB
sql = "SELECT * FROM CHF.physicalproperties"
cur.execute(sql)
result = cur.fetchall()
pp_tb = pd.DataFrame(result)

cur.close()
del result, sql, connect # delete quiry log

print("Loading database completed. data size: {}".format(len(raw_tb)))
print("Loading database completed. data size: {}".format(len(pp_tb)))

In [ ]:
# Calculation of physical properties

# if physical properties can't calculated, the other step is need (interpolation based on the physical properties table) = Sodium
prop_tb = raw_tb.copy()

"""
R113, R114, R21 can't use viscosity, thermal conductivity model -> re-mapping these models based on interpolation tables
R12 Pressure range : 0.242551~4.1361e+06 Pa (P, Q Flash)
H2O Pressure range : 611.655~2.2064e+07 Pa (P, Q Flash)
ValueError: For now, we don't support T [273.545 K] below Tmin(saturation) [276.969 K] : PropsSI("H","T",273.545,"P",6894757.1,"D2O") -> convert to D2O -> H2O
"""

# start the timer
start_time = time.time()
print('START TIME :',str(datetime.now())[10:19] )

# Set osv correlation method
m_idx = 1

# compute physical properties
for i in range(0, len(prop_tb)):
    # calculate default parameter
    prop_tb.loc[i, 'qt'] = prop_tb.loc[i, 'qi'] + prop_tb.loc[i, 'qo']
    # initialize heat flux
    try:
        if prop_tb.loc[i, 'hs'] == 1:
            prop_tb.loc[i, 'qav'] = prop_tb.loc[i, 'qi']
        elif prop_tb.loc[i, 'hs'] == 2:
            prop_tb.loc[i, 'qav'] = prop_tb.loc[i, 'qo']
        else:
            prop_tb.loc[i, 'qav'] = 1/(0.5*(1/prop_tb.loc[i, 'qi']+1/prop_tb.loc[i, 'qo']))
    except:
        prop_tb.loc[i, 'qav'] = prop_tb.loc[i, 'qt']

    prop_tb.loc[i, 'mass'] = round(PropsSI(prop_tb.loc[i, 'refri'], 'molemass') * 1e3,6)
    prop_tb.loc[i, 'de'], prop_tb.loc[i, 'ph'], prop_tb.loc[i,'af'] = pro.cal_de(prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'lh'])
    prop_tb.loc[i, 'lpd'] = prop_tb.loc[i, 'lh']/prop_tb.loc[i, 'dhy'] # Lh / dh
    prop_tb.loc[i, 'lpde'] = prop_tb.loc[i, 'lh']/prop_tb.loc[i, 'de'] # Lh / de
    prop_tb.loc[i, 'qr'] = prop_tb.loc[i, 'qi']/(prop_tb.loc[i, 'qi'] + prop_tb.loc[i, 'qo'])

    # calculate physical properties at saturation point
    prop_tb.loc[i,'tsat'] = round(PropsSI('T', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'pcrit'] = round(PropsSI(prop_tb.loc[i, 'refri'],'pcrit') * 10**-5, 12)
    prop_tb.loc[i,'tcrit'] = round(PropsSI('Tcrit', prop_tb.loc[i, 'refri']), 12)
    prop_tb.loc[i,'rhocrit'] = round(PropsSI('rhocrit', prop_tb.loc[i, 'refri']), 12)
    #prop_tb.loc[i,'rhoredu'] = round(PropsSI(prop_tb.loc[i, 'refri'], 'rhoreduce'), 12)
    #prop_tb.loc[i,'acf'] = round(PropsSI('accentric', prop_tb.loc[i, 'refri']), 12)
    prop_tb.loc[i,'rdcp'] = round(prop_tb.loc[i, 'pre']/prop_tb.loc[i, 'pcrit'],12)
    prop_tb.loc[i,'rhof'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12)
    prop_tb.loc[i,'rhov'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12)
    
    # set coefficient parameter
    if prop_tb.loc[i, 'refri'] == 'R113' or prop_tb.loc[i, 'refri'] == 'R114' or prop_tb.loc[i, 'refri'] == 'R21':
        prop_tb.loc[i,'muf'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['muf'], prop_tb.loc[i, 'pre'], method='linear')
        prop_tb.loc[i,'muv'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['muv'], prop_tb.loc[i, 'pre'], method='linear')
        prop_tb.loc[i, 'kf'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['kf'], prop_tb.loc[i, 'pre'], method='linear')
        prop_tb.loc[i, 'kv'] = griddata(pp_tb[pp_tb['refri'] == prop_tb.loc[i, 'refri']]['p'], pp_tb['kv'], prop_tb.loc[i, 'pre'], method='linear')
    else:
        prop_tb.loc[i,'muf'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12)
        prop_tb.loc[i,'muv'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12)
        prop_tb.loc[i, 'kf'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12) # [W/m/K]
        prop_tb.loc[i, 'kv'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12) # [W/m/K]
    
    prop_tb.loc[i,'hfo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i,'hvo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
    prop_tb.loc[i, 'lam'] = round((prop_tb.loc[i, 'hvo'] - prop_tb.loc[i,'hfo']),12) # [kJ/kg]

    # calculate physical propoerties of water at same conditions
    prop_tb.loc[i,'wtsat'] = round(PropsSI('T', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wpcrit'] = round(PropsSI(prop_tb.loc[i, 'refri'],'pcrit') * 10**-5, 12)
    prop_tb.loc[i,'wrdcp'] = round(prop_tb.loc[i, 'pre']/prop_tb.loc[i,'pcrit'],12)
    prop_tb.loc[i,'wrhof'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wrhov'] = round(PropsSI('D', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12)
    prop_tb.loc[i,'wmuf'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12)
    prop_tb.loc[i,'wmuv'] = round(PropsSI('V', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12)
    prop_tb.loc[i,'whfo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O')*1e-3,12) #[kJ/kg]
    prop_tb.loc[i,'whvo'] = round(PropsSI('H', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O')*1e-3,12) #[kJ/kg]
    prop_tb.loc[i, 'wlam'] = round((prop_tb.loc[i, 'whvo'] - prop_tb.loc[i,'whfo']),12) # [kJ/kg]

    # calculate flow parameters
    if pd.isna(prop_tb.loc[i, 'vf']):
        prop_tb.loc[i, 'vf'] = round(prop_tb.loc[i, 'gre'] / prop_tb.loc[i, 'rhof'], 12) # [m/s]
    else:
        if pd.isna(prop_tb.loc[i, 'gre']):
            prop_tb.loc[i, 'gre'] = round(prop_tb.loc[i, 'vf']*prop_tb.loc[i, 'rhof'], 12) # [kg/m2s]
    prop_tb.loc[i, 'cpf']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'cpv']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, prop_tb.loc[i, 'refri']),12) # [J/kgK]
    prop_tb.loc[i, 'sigma'] = round(PropsSI('I', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, prop_tb.loc[i, 'refri']),12) # [N/m]
    
    prop_tb.loc[i, 'wcpf']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12) # [J/kgK]
    prop_tb.loc[i, 'wcpv']   = round(PropsSI('C', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12) # [J/kgK]
    prop_tb.loc[i, 'wsigma'] = round(PropsSI('I', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12) # [N/m]
    prop_tb.loc[i, 'wkf'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 0, 'H2O'),12) # [W/m/K]
    prop_tb.loc[i, 'wkv'] = round(PropsSI('L', 'P', prop_tb.loc[i, 'pre'] * 1e5, 'Q', 1, 'H2O'),12) # [W/m/K]
    
    # enthin is empty enthin = hsat - he
    # Fill Na related to inlet subcooling
    if pd.isna(prop_tb.loc[i, 'xi']):
        # Xi is NaN
        if pd.isna(prop_tb.loc[i, 'enthin']):
            # Xi, enthin is NaN
            if pd.isna(prop_tb.loc[i, 'degtin']):
                # Xi, enthin, degtin is NaN
                if pd.isna(prop_tb.loc[i, 'tin']):
                    # Xi, enthin, degtin, and tin is NaN
                    #print("idx: {} and {}th data is not inlet condition".format(prop_tb.loc[i, 'idx'], i))
                    prop_tb.loc[i, 'xi'] = pro.cal_xi(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xe'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 0)
                    
                    prop_tb.loc[i, 'enthin'] = round(prop_tb.loc[i, 'xi']*prop_tb.loc[i, 'lam'], 12)
                    prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
                    prop_tb.loc[i, 'degtin'] = round((prop_tb.loc[i, 'enthin']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
                    prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] - 273.15 # degC
                else:
                    # ONLY tin exist
                    prop_tb.loc[i, 'hin'] = round(PropsSI('H', 'T', prop_tb.loc[i, 'tin'] +273.15, 'P', prop_tb.loc[i, 'pre'] * 1e5, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
                    prop_tb.loc[i, 'enthin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'hin'], 6)  # [kJ/kg]
                    prop_tb.loc[i, 'degtin'] = round(prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'tin'], 6) # [degC]
                    prop_tb.loc[i, 'xi'] = round(-prop_tb.loc[i, 'enthin']/prop_tb.loc[i, 'lam'], 12)
            else:
                # ONLY degtin exist
                prop_tb.loc[i, 'xi'] = round(-prop_tb.loc[i, 'cpf']*prop_tb.loc[i, 'degtin']*1e-3/prop_tb.loc[i, 'lam'],6) #[-]
                prop_tb.loc[i, 'enthin'] = round(prop_tb.loc[i, 'cpf'] * prop_tb.loc[i, 'degtin']*1e-3, 6) # [kJ/kg]
                prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
                prop_tb.loc[i, 'tin'] = round(prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] - 273.15, 6) # degC
        else:
            # enthin exist
            prop_tb.loc[i, 'xi'] = round(-prop_tb.loc[i, 'enthin']/prop_tb.loc[i, 'lam'], 6)
            prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]            
            prop_tb.loc[i, 'degtin'] = round((prop_tb.loc[i, 'enthin']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
            prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] - 273.15 # degC
    else:
        # Xi exist
        prop_tb.loc[i, 'enthin'] = round(-prop_tb.loc[i, 'xi']*prop_tb.loc[i, 'lam'], 12)
        prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
        prop_tb.loc[i, 'degtin'] = round((prop_tb.loc[i, 'enthin']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
        prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] - prop_tb.loc[i, 'degtin'] - 273.15 # degC

    # enthout is empty enthout = he - hsat
    # Fill Na related to outlet subcooling
    if pd.isna(prop_tb.loc[i, 'xe']):
        # Xe is NaN
        if pd.isna(prop_tb.loc[i, 'enthout']):
            # Xe, enthout is NaN
            if pd.isna(prop_tb.loc[i, 'degtout']):
                # Xe, enthout, degtout is NaN
                if pd.isna(prop_tb.loc[i, 'tout']):
                    # Xe, enthout, degtout, and tout is NaN
                    #print("idx: {} and {}th data is not outlet condition".format(prop_tb.loc[i, 'idx'], i))
                    prop_tb.loc[i, 'xe'] = pro.cal_xe(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xi'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 0)
                    prop_tb.loc[i, 'enthout'] = round(prop_tb.loc[i, 'xe']*prop_tb.loc[i, 'lam'], 12)
                    prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'enthout'] + prop_tb.loc[i, 'hfo'], 12) # [kJ/kg]
                    prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
                    prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'degtout'] + prop_tb.loc[i, 'tsat'] - 273.15 # degC
                else:
                    # ONLY tout exist
                    prop_tb.loc[i, 'hout'] = round(PropsSI('H', 'T', prop_tb.loc[i, 'tout'] +273.15, 'P', prop_tb.loc[i, 'pre'] * 1e5, prop_tb.loc[i, 'refri'])*1e-3,12) #[kJ/kg]
                    prop_tb.loc[i, 'enthout'] = prop_tb.loc[i, 'hout'] - prop_tb.loc[i, 'hfo']  # [kJ/kg]
                    prop_tb.loc[i, 'degtout'] = prop_tb.loc[i, 'tout'] + 273.15 - prop_tb.loc[i, 'tsat'] # [degC]
                    prop_tb.loc[i, 'xe'] = round(prop_tb.loc[i, 'enthout']/prop_tb.loc[i, 'lam'],12)
            else:
                # ONLY degtout exist
                prop_tb.loc[i, 'xe'] = round(prop_tb.loc[i, 'cpf']*prop_tb.loc[i, 'degtout']*1e-3/prop_tb.loc[i, 'lam'],6) #[-]
                prop_tb.loc[i, 'enthout'] = round(prop_tb.loc[i, 'xe']*prop_tb.loc[i, 'lam'], 12)
                prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'enthout'] + prop_tb.loc[i, 'hfo'], 12) # [kJ/kg]
                prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'degtout'] + prop_tb.loc[i, 'tsat'] - 273.15 # degC
        else:
            # enthout exist
            prop_tb.loc[i, 'xe'] = round(prop_tb.loc[i, 'enthout']/prop_tb.loc[i, 'lam'], 6)
            prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'enthout'] + prop_tb.loc[i, 'hfo'], 12) # [kJ/kg]
            prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
            prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'degtout'] + prop_tb.loc[i, 'tsat'] - 273.15 # degC
            
    else:
        # Xe exist
        prop_tb.loc[i, 'enthout'] = round(prop_tb.loc[i, 'xe']*prop_tb.loc[i, 'lam'], 12)
        prop_tb.loc[i, 'hout'] = round(prop_tb.loc[i, 'enthout'] + prop_tb.loc[i, 'hfo'], 12) # [kJ/kg]
        prop_tb.loc[i, 'degtout'] = round((prop_tb.loc[i, 'enthout']*1e3)/prop_tb.loc[i, 'cpf'], 12) # degC
        prop_tb.loc[i, 'tout'] = prop_tb.loc[i, 'degtout'] + prop_tb.loc[i, 'tsat'] - 273.15 # degC
    
    # fill inlet/oulet data
    if pd.isna(prop_tb.loc[i, 'xi']):
        prop_tb.loc[i, 'xi'] = pro.cal_xi(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xe'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 0)
        prop_tb.loc[i, 'enthin'] = round(prop_tb.loc[i, 'xi']*prop_tb.loc[i, 'lam'], 12)
        prop_tb.loc[i, 'hin'] = round(prop_tb.loc[i, 'hfo'] - prop_tb.loc[i, 'enthin'], 12) # [kJ/kg]
        prop_tb.loc[i, 'degtin'] = round((prop_tb.loc[i, 'enthin']*1e-3)/prop_tb.loc[i, 'cpf'], 12) # degC
        prop_tb.loc[i, 'tin'] = prop_tb.loc[i, 'tsat'] -273.15 - prop_tb.loc[i, 'degtin']  # degC
    else:
        pass
    
    #prop_tb.loc[i,'zc'] = round(PropsSI('Z', 'T', prop_tb.loc[i, 'tsat'], prop_tb.loc[i,'rhof'], prop_tb.loc[i, 'refri']), 12)

    # Check heat balance
    prop_tb.loc[i,'hval'] = pro.cal_xe(prop_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xi'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 1)
    prop_tb.loc[i,'param_hval'] = round(np.abs((prop_tb.loc[i ,'xe']-prop_tb.loc[i, 'hval'])/prop_tb.loc[i, 'hval']), 6)
    if prop_tb.loc[i, 'param_hval'] < 0.05:
        prop_tb.loc[i, 'yn_hval'] = 100
    else:
        prop_tb.loc[i, 'yn_hval'] = 110

# Calculate dimensionless parameters
for i, row in prop_tb.iterrows():

    # Calculate dimensionless number
    prop_tb.loc[i, 'pe'] = round(
        pro.cal_pe(prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 're'] = round(pro.cal_re(prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'muf']), 6)
    prop_tb.loc[i, 'we'] = round(
        pro.cal_we(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'bd'] = round(
        pro.cal_bd(prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'sigma']), 6)
    prop_tb.loc[i, 'bo'] = round(pro.cal_bo(prop_tb.loc[i, 'qav'],prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'gre']), 6) # Boiling number
    prop_tb.loc[i, 'pr'] = round(pro.cal_pr(prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf']), 6)
    prop_tb.loc[i, 'prv'] = round(pro.cal_pr(prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'muv'], prop_tb.loc[i, 'kv']), 6)
    prop_tb.loc[i, 'ca'] = round(
        pro.cal_ca(prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'rhof']), 6)
    prop_tb.loc[i, 'lc'] = round(np.sqrt(prop_tb.loc[i, 'sigma']/(9.8*(prop_tb.loc[i, 'rhof']-prop_tb.loc[i, 'rhov']))), 6)
    # Calculate old_xosv (Initial condition)
    if m_idx == 1:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_SZ(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
    else:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'vf'])
      
# stop the timer
end_time = time.time()
print('END TIME :',str(datetime.now())[10:19])

# calculate the elapsed time
elapsed_time = end_time - start_time

# print the elapsed time
print("Elapsed time to compute the geometrical or dimensionless parameters: {:.2f} mins".format(elapsed_time/60))

# remove local parameters
del start_time, end_time, elapsed_time

In [ ]:
# Set the features
columns = ['pre', 'qav', 'gre', 'dhy', 'lh', 'xi', 'xe', 'lpd', 're', 'pr', 'prv', 'qr']

# Initialize a dataframe for saving the boundary values
boundaries_df = pd.DataFrame(columns=['col', 'gro', 'min', 'max'])

# Initialized previous maximum value
prev_max = 0

# K-means clustering and storing group indices and boundaries
for col in columns:
    X = prop_tb[[col]]
    kmeans = KMeans(n_clusters=25, random_state=0, n_init=5)
    labels = kmeans.fit_predict(X)
    prop_tb[f'{col}_Group'] = labels + 1  # Group indices start from 1
    
    # Calculate and store the boundaries of each group
    centers = kmeans.cluster_centers_.flatten()
    centers.sort()
    for i, center in enumerate(centers):
        if i > 0:
            new_min = prev_max + 0.001
            new_max = min(center, new_min + (center - new_min) / 2)  # Adjust the maximum value
            new_boundary = pd.DataFrame({
                'col': [col],
                'gro': [i],
                'min': [new_min],
                'max': [new_max]
            })
        else:
            new_min = 0
            new_max = center
            new_boundary = pd.DataFrame({
                'col': [col],
                'gro': [i],
                'min': [new_min],
                'max': [new_max]
            })
        boundaries_df = pd.concat([boundaries_df, new_boundary], ignore_index=True)
        prev_max = new_max

# Sort boundaries_df by 'Min' values in ascending order
boundaries_df = boundaries_df.sort_values('min', ascending=True)

# Update prop_tb with new group numbers based on boundaries_df
for col in columns:
    group_col = f'{col}_Group'
    for i, row in boundaries_df[boundaries_df['col'] == col].iterrows():
        group_mask = (prop_tb[col] >= row['min']) & (prop_tb[col] <= row['max'])
        prop_tb.loc[group_mask, group_col] = row['gro']

In [ ]:
boundaries_df

In [ ]:
# create session of database engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')

# Export results of physical property to MariaDB
prop_tb.to_sql('prop_chf_tb', engine, if_exists='replace')
boundaries_df.to_sql('boundaries_tb', engine, if_exists='replace')

del engine

In [ ]:
# DSM Method (Local condition hypothesis)

dsm_tb = prop_tb.copy()

# Set osv correlation method
m_idx = 1

for i, row in dsm_tb.iterrows():
    # Step 1 calculate Xc
    dsm_tb.loc[i, 'xc'] = pro.cal_xe(dsm_tb.loc[i, 'qi'], prop_tb.loc[i, 'qo'], dsm_tb.loc[i, 'doi'], dsm_tb.loc[i, 'dio'], dsm_tb.loc[i, 'geo'], dsm_tb.loc[i, 'hs'], dsm_tb.loc[i, 'gre'], dsm_tb.loc[i, 'xi'], dsm_tb.loc[i, 'lh'], dsm_tb.loc[i, 'lam'], ch = 0)

    # Step 1 calculate osv
    if m_idx == 1:
        dsm_tb.loc[i, 'dtsz'], dsm_tb.loc[i, 'xosv'] = mod.cal_SZ(dsm_tb.loc[i, 'qi']+dsm_tb.loc[i, 'qo'], dsm_tb.loc[i, 'rhof'], dsm_tb.loc[i, 'dhy'], dsm_tb.loc[i, 'gre'], dsm_tb.loc[i, 'cpf'], dsm_tb.loc[i, 'kf'], dsm_tb.loc[i, 'pe'], dsm_tb.loc[i, 'lam'])
    else:
        dsm_tb.loc[i, 'dtsz'], dsm_tb.loc[i, 'xosv'] = mod.cal_Levy(dsm_tb.loc[i, 'sigma'], dsm_tb.loc[i, 'dhy'], dsm_tb.loc[i, 'rhof'], dsm_tb.loc[i, 'muf'], dsm_tb.loc[i, 'kf'], dsm_tb.loc[i, 're'], dsm_tb.loc[i, 'pr'], dsm_tb.loc[i, 'cpf'], dsm_tb.loc[i, 'gre'], dsm_tb.loc[i, 'qi']+dsm_tb.loc[i, 'qo'], dsm_tb.loc[i, 'lam'], dsm_tb.loc[i, 'vf'])
    
    # Step 2 calculate Xt
    dsm_tb.loc[i, 'xt'] = round(mod.cal_xt(dsm_tb.loc[i, 'xi'], dsm_tb.loc[i, 'xosv'], dsm_tb.loc[i, 'xc'], dsm_tb.loc[i, 'xc']),6)
    dsm_tb.loc[i, 'alpha'], dsm_tb.loc[i, 'gamma'], dsm_tb.loc[i, 'zxt'], dsm_tb.loc[i, 'qcal'] = mod.calCHFDeng(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], dsm_tb.loc[i, 'xt'])
    dsm_tb.loc[i, 'qval'] = round((dsm_tb.loc[i, 'qcal'] - dsm_tb.loc[i, 'qav'])/dsm_tb.loc[i, 'qav'], 4)

In [ ]:
# Save result of DSM method
# create engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')

# export the log- and result-table to MariaDB
dsm_tb.to_sql('res_dsm_tb', engine, if_exists='replace')

# delete parameter
del engine

In [ ]:
# HBM Method (Inlet condition hypothesis)
# set iniital calculation method
m_idx = 1
cal_idx = 3
mod_idx = 'Jeong'

# create log table 
log_tb = pd.DataFrame()

# set global parameters
global j
j = 0
tolerance = 0.001

# start the timer
start_time = time.time()
print('START TIME :',str(datetime.now())[10:19] )

for i, row in prop_tb.iterrows():
    if m_idx == 1:
        prop_tb.loc[i, 'mth_xosv'] = 'SZ'
    else:
        prop_tb.loc[i, 'mth_xosv'] = 'Levy'
    
    cnt = 1
    # Preparing alpha and gamma
    prop_tb.loc[i, 'log_id'] = int(i) # Set foreign key to log-table

    # calculate original Xosv from experimental CHF heat flux
    if m_idx == 1:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_SZ(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
    else:
        prop_tb.loc[i, 'org_dtsz'], prop_tb.loc[i, 'org_xosv'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'vf'])
    
    while 1:
               
        # Ref: Original algorithm
            # calculate initial Xt from qc correlation using qexp
            # calculate Xo using qexp = - StGlamXo
            # calculate Xt from rate equation
            # compare oldXt vs. new Xt

        # Step 0: Initialize Xt when the first iteration step
        if cnt == 1:
            # Prepare : log-table
            log_tb.loc[j, 'run_id'] = int(i)
            log_tb.loc[j, 'org_xi'] = prop_tb.loc[i, 'xi']
            log_tb.loc[j, 'org_xe'] = prop_tb.loc[i, 'xe']
            # initialize heat flux
            log_tb.loc[j, 'org_q'] = prop_tb.loc[i, 'qav']
            log_tb.loc[j, 'yn_hval'] = prop_tb.loc[i, 'yn_hval']
            if log_tb.loc[j, 'org_xe'] < 0:
                log_tb.loc[j, 'old_xt'] = 0.001
            else:
                log_tb.loc[j, 'old_xt'] = log_tb.loc[j, 'org_xe']
        else:
            pass

        # Step 2: Calculate qc,old geo, hs, doi, dio, dh, p, pcrit, g, q, muf, muv, rhof, rhov, cpf, cpv, v, lam, mass, tcrit, xt_cal=0.5)
        #log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'old_qcal'] = mod.calCHFJeong(prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'pre'], prop_tb.loc[i, 'pcrit'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'muv'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'mass'], prop_tb.loc[i, 'tcrit'], log_tb.loc[j, 'old_xt'])

        log_tb.loc[j, 'alpha'], log_tb.loc[j, 'gamma'], log_tb.loc[j, 'zxt'], log_tb.loc[j, 'old_qcal'] = mod.calCHFDeng(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], log_tb.loc[j, 'old_xt'])
        
        # Step 3: Calculate old_xosv
        if m_idx == 1:
            log_tb.loc[j, 'old_dtsz'], log_tb.loc[j, 'old_xosv'] = mod.cal_SZ(log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 'pe'], prop_tb.loc[i, 'lam'])
        else:
            log_tb.loc[j, 'old_dtsz'], log_tb.loc[j, 'old_xosv'] = mod.cal_Levy(prop_tb.loc[i, 'sigma'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'kf'], prop_tb.loc[i, 're'], prop_tb.loc[i, 'pr'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'gre'], log_tb.loc[j, 'old_qcal'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'vf'])

        # Step 4: Calculate old_xe
        log_tb.loc[j, 'old_xe'] = pro.cal_xe(log_tb.loc[j, 'old_qcal'], 0, prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'xi'], prop_tb.loc[i, 'lh'], prop_tb.loc[i, 'lam'], ch = 1)

        # Step 2: Calculate old_xt (initialization)
        log_tb.loc[j, 'new_xt'] = round(mod.cal_xt(log_tb.loc[j, 'org_xi'], log_tb.loc[j, 'old_xosv'], log_tb.loc[j, 'org_xe'], log_tb.loc[j, 'old_xe']),6)

        # Step 3: Loop control
        cnt += 1 # Increase limitation

        # Step 4: Valudation
        val = abs((log_tb.loc[j, 'new_xt'] - log_tb.loc[j, 'old_xt'])/log_tb.loc[j, 'old_xt'])

        if val <= tolerance:
            # Step 2: Calculate old_xt (initialization)
            prop_tb.loc[i, 'cal_xt'] = round(float(log_tb.loc[j, 'new_xt']),6)
            prop_tb.loc[i, 'cal_xe'] = round(log_tb.loc[j, 'old_xe'], 4)
            prop_tb.loc[i, 'cal_xosv'] = round(log_tb.loc[j, 'old_xosv'], 4)
            prop_tb.loc[i, 'converged'] = 100
            prop_tb.loc[i, 'alpha'], prop_tb.loc[i, 'gamma'], prop_tb.loc[i, 'zxt'], prop_tb.loc[i, 'qcal'] = mod.calCHFDeng(prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'rdcp'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'gre'], log_tb.loc[j, 'new_xt'])
            #prop_tb.loc[i, 'alpha'], prop_tb.loc[i, 'gamma'], prop_tb.loc[i, 'zxt'], prop_tb.loc[i, 'qcal'] = mod.calCHFJeong(prop_tb.loc[i, 'geo'], prop_tb.loc[i, 'hs'], prop_tb.loc[i, 'doi'], prop_tb.loc[i, 'dio'], prop_tb.loc[i, 'dhy'], prop_tb.loc[i, 'pre'], prop_tb.loc[i, 'pcrit'], prop_tb.loc[i, 'gre'], prop_tb.loc[i, 'qav'], prop_tb.loc[i, 'muf'], prop_tb.loc[i, 'muv'], prop_tb.loc[i, 'rhof'], prop_tb.loc[i, 'rhov'], prop_tb.loc[i, 'cpf'], prop_tb.loc[i, 'cpv'], prop_tb.loc[i, 'vf'], prop_tb.loc[i, 'lam'], prop_tb.loc[i, 'mass'], prop_tb.loc[i, 'tcrit'], log_tb.loc[j, 'new_xt'])
            prop_tb.loc[i, 'qval'] = round((prop_tb.loc[i, 'qcal'] - log_tb.loc[j, 'org_q'])/log_tb.loc[j, 'org_q'], 4)
            if abs(prop_tb.loc[i, 'qval']) > 0.20:
                if cnt > 500:
                    print("Data run number : {:d} isn't converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                    del cnt
                    break
                else:
                    log_tb.loc[j+1, 'old_xt'] = 0.99* log_tb.loc[j, 'old_xt'] + 0.01*log_tb.loc[j, 'new_xt']
                    j += 1
                    continue
            else:
                print("Data run number : {:d} is converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                j += 1
                del cnt
                break
        else:
            if cnt > 500:
                prop_tb.loc[i, 'cal_xt'] = round(float(log_tb.loc[j, 'new_xt']),6)
                prop_tb.loc[i, 'cal_xe'] = log_tb.loc[j, 'old_xe']
                prop_tb.loc[i, 'cal_xosv'] = log_tb.loc[j, 'old_xosv']
                prop_tb.loc[i, 'converged'] = 110
                prop_tb.loc[i, 'qval'] = ((prop_tb.loc[i, 'qcal'] - log_tb.loc[j, 'org_q'])/log_tb.loc[j, 'org_q'])
                if abs(prop_tb.loc[i, 'qval']) < 0.20:
                    print("Data run number : {:d} is converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                    prop_tb.loc[i, 'alpha'] = log_tb.loc[j, 'alpha']
                    prop_tb.loc[i, 'gamma'] = log_tb.loc[j, 'gamma']
                    prop_tb.loc[i, 'zxt'] = log_tb.loc[j, 'zxt']
                    prop_tb.loc[i, 'qcal'] = log_tb.loc[j, 'old_qcal']
                    j += 1
                    del cnt
                    break
                else:
                    print("Data run number : {:d} isn't converged. Xosv: {:.4f}, Xe: {:.4f}, Xt: {:.4f}, qexp: {:.4f}, qcal: {:.4f}, and qval: {:.2f}".format(i, prop_tb.loc[i, 'cal_xosv'], prop_tb.loc[i, 'xe'], prop_tb.loc[i,'cal_xt'], prop_tb.loc[i,'qav'], prop_tb.loc[i,'qcal'], prop_tb.loc[i, 'qval']))
                    prop_tb.loc[i, 'alpha'] = log_tb.loc[j, 'alpha']
                    prop_tb.loc[i, 'gamma'] = log_tb.loc[j, 'gamma']
                    prop_tb.loc[i, 'zxt'] = log_tb.loc[j, 'zxt']
                    prop_tb.loc[i, 'qcal'] = log_tb.loc[j, 'old_qcal']
                    j += 1
                    del cnt
                    break
            else:
                # Step 5: Recurrsive state
                log_tb.loc[j+1, 'old_xt'] = 0.9* log_tb.loc[j, 'old_xt'] + 0.1*log_tb.loc[j, 'new_xt']
                j += 1
                continue

# replace nan value to 0
prop_tb.fillna(0, inplace=True)

# stop the timer
end_time = time.time()
print('END TIME :',str(datetime.now())[10:19])

# calculate the elapsed time
elapsed_time = end_time - start_time

# print the elapsed time
print("Elapsed time to compute the physical properties: {:.2f} mins".format(elapsed_time/60))

# delete parameters
del i, j, tolerance, start_time, end_time, elapsed_time

In [ ]:
# create engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(json_res["user"][1],json_res["password"][1],json_res["host"][1],json_res["port"][1],"CHF"), echo=False, encoding='utf-8')
log_tb = log_tb.replace([np.inf, -np.inf], 0)
prop_tb = prop_tb.replace([np.inf, -np.inf], 0)

# export the log- and result-table to MariaDB
log_tb.to_sql('deng_log_chf_tb', engine, if_exists='replace')
prop_tb.to_sql('deng_res_chf_tb', engine, if_exists='replace')

# delete parameter
del engine